<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sodapy

In [66]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_squared_log_error
%matplotlib inline

In [67]:
def get_random_samples(client, num_samples=200, sample_size=1000, verbose=False):
    
    start = time.time()

    # Perform a $select=count(*) query to determine how large the set is
    results = client.get("m6dm-c72p", select='count(*)' )
    total_rows = int(results[0].get('count', 0))
    row_indices = np.arange(0, total_rows, sample_size)
    results = []

    # Use rand() locally to come up with some offsets
    sample_offsets = np.random.choice(row_indices, size=num_samples, replace=False)


    # Use $limit and $offset in conjunction with a stable $order to pick out individual records. 
    # Ex: $order=facility_id&$limit=1&$offset=<some rand() number>
    for i, offset in enumerate (sample_offsets):

        if verbose:
            print(f'Sample {i}: offset={offset},sample_size={sample_size}')
            print('Pure Python time:', time.time() - start, 'sec.')
        results.extend(client.get("m6dm-c72p", order='trip_id', limit=sample_size, offset=offset, 
                                                  select='''trip_id, trip_start_timestamp, pickup_community_area, fare, tip, trip_total'''))
        
    if verbose:
        print('Pure Python time:', time.time() - start, 'sec.')

    return results

In [61]:
# %run ../python_files/utils
# %run ../python_files/data_cleaning

# df = get_trip_records(100000)
# df = clean_data(df)
# df

In [62]:
# samples = get_random_samples(client, verbose=True )
# samples[:10]

In [64]:
# samples_df = pd.DataFrame.from_records(samples)
# samples_df

In [68]:
client = Socrata('data.cityofchicago.org',
             'Tk6RhuGAFvF9P4ehsysybj3IW',
             username="mokome@gmail.com",
             password="Ch1cago!!")

client.timeout = 10000

results = client.get("m6dm-c72p", limit=1000000, select='trip_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tip, additional_charges, trip_total' )

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,231604c8d620a75e15e9b66f390a618b8024154a,2018-12-05T11:15:00.000,2018-12-05T11:15:00.000,579,1.20805789824,28,8,5,2,2.5,9.5
1,231604ce194695272c6247d704b14ebab62bb647,2018-11-04T00:15:00.000,2018-11-04T00:30:00.000,1329,8.27805517616,NaN,22,20,0,2.5,22.5
2,231604cf4a57f8d2c0db6691e7a95c9b946705d4,2018-12-21T00:30:00.000,2018-12-21T00:45:00.000,1445,10.1542800004925,8,77,5,0,2.5,7.5
3,231605271d5f4528b53c57ed8bf20791ba7afbd2,2019-03-10T23:15:00.000,2019-03-10T23:45:00.000,1899,17.84681830224,76,28,25,0,0,25
4,231605d462fc9f6fd23a8b65a3c5cb1241a97e8d,2018-12-29T13:15:00.000,2018-12-29T13:30:00.000,1081,4.47041427040939,8,24,10,0,2.5,12.5
...,...,...,...,...,...,...,...,...,...,...,...
999995,2ac6367d77fd712a5226b2e56146fc332f6512b6,2018-11-13T07:45:00.000,2018-11-13T08:15:00.000,2160,11.40629959,35,5,0,0,0.67,0.67
999996,2ac63697ad06080b4e55f08ff46d2353728ba460,2019-03-29T08:00:00.000,2019-03-29T08:15:00.000,1126,2.02771295296504,24,32,7.5,0,2.55,10.05
999997,2ac636e08ed5376ea0867fb5e72e2a998113c073,2018-12-14T02:45:00.000,2018-12-14T03:15:00.000,1652,5.46360341632,53,53,7.5,0,2.5,10
999998,2ac636f90011cce68754cb86f0b86b27ff816d41,2019-02-16T14:00:00.000,2019-02-16T14:00:00.000,105,0.34266236753139,28,28,2.5,0,2.55,5.05


In [ ]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].fillna('0')
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df['start_date_plus_hour'] = df['trip_start_timestamp'].apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

weather_df = pd.read_csv('../data/chicago_weather.csv')
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))
weather_df['start_date_plus_hour'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df = weather_df.rename(columns={'icon': 'precip'})

# def set_precip(precip):
#     if precip not in ['rain', 'snow']: 
#         precip = 'clear'

#     return precip

# weather_df['precip'] = weather_df['precip'].apply(set_precip)
precip_df = weather_df[['start_date_plus_hour', 'precip', 'apparentTemperature']]

df = df.merge(precip_df, how='left', on='start_date_plus_hour')
df.head()

In [9]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 16 columns):
trip_id                  100000 non-null object
trip_start_timestamp     100000 non-null datetime64[ns]
trip_end_timestamp       100000 non-null datetime64[ns]
trip_seconds             100000 non-null int64
trip_miles               99993 non-null float64
pickup_community_area    93718 non-null object
fare                     100000 non-null float64
tip                      100000 non-null float64
additional_charges       100000 non-null float64
trip_total               100000 non-null float64
start_weekday            100000 non-null int64
start_hour               100000 non-null int64
start_time_block         100000 non-null int64
start_date_plus_hour     100000 non-null datetime64[ns]
precip                   100000 non-null object
apparentTemperature      100000 non-null float64
dtypes: datetime64[ns](3), float64(6), int64(4), object(3)
memory usage: 13.0+ MB


In [46]:
# Limit to the columns we are interested in: 
# 'apparentTemperature', 'start_weekday', 'start_hour', (OR 'start_time_block'), 'pickup_community_area' 

columns_to_use = ['apparentTemperature', 'start_weekday', 'start_hour', 'pickup_community_area']
columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]

# Use dependent variables listed above to predict the independent variable: 'trip_total' OR 'fare'
X = df.drop(columns=columns_to_drop)
y = df['trip_total']


# deal with any null values
X['apparentTemperature']=X['apparentTemperature'].fillna(X['apparentTemperature'].median())
X['pickup_community_area']=X['pickup_community_area'].fillna('0')


# Split out continuous & categorical variables
cont_cols = ['apparentTemperature']
cat_cols = [ col for col in columns_to_use if col not in cont_cols ]

enc = OneHotEncoder()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

X_train_cont = X_train[cont_cols]
X_train_cat = X_train[cat_cols]

# ss = StandardScaler()
# X_train_cont = pd.DataFrame(ss.fit_transform(X_train_cont))

# Transform training set
X_train_enc = enc.fit_transform(X_train_cat, y_train)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train_cat.columns)
X_train_cat = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)

# Combine categorical and continuous features into the final dataframe
X_train = pd.concat([X_train_cont, X_train_cat], axis=1)
X_train_const = sm.add_constant(X_train)

# Fit model & show summary
model = sm.OLS(y_train,X_train_const).fit()
model.summary()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     267.1
Date:                Thu, 23 Jan 2020   Prob (F-statistic):               0.00
Time:                        02:34:58   Log-Likelihood:            -2.8590e+05
No. Observations:               75000   AIC:                         5.720e+05
Df Residuals:                   74908   BIC:                         5.728e+05
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         10.2484      0.142     72.426      0.000       9.971      10.526
apparentTemperature            0.0327      0.003     11.319      0.000       0.027       0.038
start_weekday_0.0              1.7009      0.108     15.776      0.000       1.490       1.912
start_weekday_1.0              0.8319      0.107      7.794      0.000       0.623       1.041
start_weekday_2.0              1.1570      0.107     10.846      0.000       0.948       1.366
start_weekday_3.0              1.3938      0.101     13.776      0.000       1.195       1.592
start_weekday_4.0              1.8596      0.095     19.626      0.000       1.674       2.045
start_weekday_5.0              1.5121      0.091     16.700      0.000       1.335       1.690
start_weekday_6.0              1.7931      0.103     17.354      0.000       1.591       1.996
start_time_block_0.0          -0.3823      0.141     -2.708      0.007      -0.659      -0.106
start_time_block_1.0           5.5915      0.184     30.346      0.000       5.230       5.953
start_time_block_2.0           2.7500      0.114     24.080      0.000       2.526       2.974
start_time_block_3.0           0.6568      0.106      6.213      0.000       0.450       0.864
start_time_block_4.0           1.0356      0.107      9.658      0.000       0.825       1.246
start_time_block_5.0           1.5941      0.100     15.972      0.000       1.398       1.790
start_time_block_6.0          -0.6508      0.095     -6.837      0.000      -0.837      -0.464
start_time_block_7.0          -0.3465      0.101     -3.416      0.001      -0.545      -0.148
pickup_community_area_0.0     13.0622      0.183     71.358      0.000      12.703      13.421
pickup_community_area_1.0     -0.3760      0.396     -0.949      0.342      -1.152       0.400
pickup_community_area_2.0     -1.7445      0.450     -3.874      0.000      -2.627      -0.862
pickup_community_area_3.0     -0.3567      0.286     -1.246      0.213      -0.918       0.204
pickup_community_area_4.0     -0.2703      0.417     -0.648      0.517      -1.089       0.548
pickup_community_area_5.0      0.3536      0.377      0.937      0.349      -0.386       1.093
pickup_community_area_6.0      0.4257      0.180      2.366      0.018       0.073       0.778
pickup_community_area_7.0     -1.0269      0.195     -5.256      0.000      -1.410      -0.644
pickup_community_area_8.0      0.7363      0.136      5.429      0.000       0.470       1.002
pickup_community_area_9.0     -0.4107      1.355     -0.303      0.762      -3.067       2.246
pickup_community_area_10.0     0.6017      0.697      0.863      0.388      -0.765       1.968
pickup_community_area_11.0    -0.5404      0.776     -0.696      0.486      -2.062       0.981
pickup_community_area_12.0     1.9399      

In [69]:
# %run ../python_files/regression

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = get_train_test_split(df, test_size=0.25)
# X_train_const = sm.add_constant(X_train)

# # Fit model & show summary
# OLS(y_train,X_train_const)


In [70]:
# LinearRegression(X_train, y_train)

In [71]:
# Fit the model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Print R2 and MSE for training set
print('Training r^2:', linreg.score(X_train, y_train))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train)))

linreg.coef_

Training r^2: 0.24481589631675804
Training MSE: 120.19252430069236


array([ 2.95571428e-02, -1.69392623e+10, -1.69392623e+10, -1.69392623e+10,
       -1.69392623e+10, -1.69392623e+10, -1.69392623e+10, -1.69392623e+10,
       -9.99833785e+09, -9.99833784e+09, -9.99833785e+09, -9.99833785e+09,
       -9.99833785e+09, -9.99833785e+09, -9.99833785e+09, -9.99833785e+09,
        1.53128239e+08,  1.53128226e+08,  1.53128225e+08,  1.53128226e+08,
        1.53128226e+08,  1.53128227e+08,  1.53128227e+08,  1.53128225e+08,
        1.53128227e+08,  1.53128227e+08,  1.53128227e+08,  1.53128226e+08,
        1.53128228e+08,  1.53128226e+08,  1.53128225e+08,  1.53128226e+08,
        1.53128225e+08,  1.53128226e+08,  1.53128225e+08,  1.53128224e+08,
        1.53128225e+08,  1.53128225e+08,  1.53128225e+08,  1.53128224e+08,
        1.53128225e+08,  1.53128225e+08,  1.53128224e+08,  1.53128224e+08,
        1.53128225e+08,  1.53128225e+08,  1.53128225e+08,  1.53128225e+08,
        1.53128228e+08,  1.53128229e+08,  1.53128228e+08,  1.53128224e+08,
        1.53128226e+08,  

In [39]:
Lasso(X_train, y_train)

Training r^2: 0.0016563287686486294
Training MSE: 158.05216084339293


Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [34]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1) # Lasso is also known as the L1 norm 
lasso.fit(X_train, y_train)

print('Training r^2:', lasso.score(X_train, y_train))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train)))

lasso.coef_

Training r^2: 0.0016563287686486294
Training MSE: 158.05216084339293


array([ 0.03126329,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.        , -0.        ,
       -0.        ,  0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        ,  0.        , -0.  

In [40]:
Ridge(X_train, y_train)

Training r^2: 0.24422052753280088
Training MSE: 119.65075974006857


Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [29]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=10) # Ridge is also known as the L2 norm
ridge.fit(X_train, y_train)

print('Training r^2:', ridge.score(X_train, y_train))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))

ridge.coef_

Training r^2: 0.24422052753280088
Training MSE: 119.65075974006857


array([ 0.03425152,  0.13272144, -0.57440017, -0.18399975, -0.05173743,
        0.43186143,  0.07358853,  0.17196594, -1.65422202,  4.28157632,
        1.4762323 , -0.66009259, -0.23871675,  0.41103615, -1.91738105,
       -1.69843236, 12.76272151, -0.3584145 , -1.66420566, -0.68058555,
       -0.464841  , -0.2946133 ,  0.34010065, -1.02033888,  0.57019683,
        0.15088632,  0.6604779 , -0.09463474,  0.9029411 , -0.65924804,
       -1.20456053, -0.26349371, -1.07032985, -0.74448602, -0.67617293,
       -2.0822037 , -1.54646591, -1.53596207, -1.1292946 , -2.17275856,
       -1.58137192, -1.65245353, -1.59816161, -2.28320311, -0.9291831 ,
       -2.15192645, -1.41643966, -1.66409653,  1.25574547,  2.62983841,
        0.97694035, -2.36086833, -0.91289014, -3.28332547, -1.3715935 ,
       -0.401022  , -2.73603591,  1.07161873, -0.4308393 , -0.47457522,
       -1.78048253, -2.205615  , -0.78472142,  0.88219358,  0.4729512 ,
       -1.80458952, -1.11047284,  1.33968932,  6.56498372,  0.75